In [ ]:
!pip install pandas
!pip install tqdm
!pip install numpy

In [ ]:
from transformers import AutoTokenizer

In [ ]:
from expressoes import lista_habilitacao
import numpy as np
import re

In [ ]:
def iterDocuments(n):
    for _ in range(n):
        yield _

In [ ]:
def get_text_split(text, iter, length=200, overlap=50, max_chunks=35):
    l_total = []
    id = []
    segment_label = []
    l_parcial = []
    n_words = len(text.split()) 
    n = n_words//(length-overlap)+1 # n is number of splits
    if n_words % (length-overlap) == 0:
        n = n-1
    if n ==0:
        n = 1
    n = min(n, max_chunks)
    id_document = next(iter)
    for w in range(n):
        if w == 0:
            l_parcial = text.split()[:length]
        else:
            l_parcial = text.split()[w*(length-overlap):w*(length-overlap) + length]
        l = " ".join(l_parcial)
        if w==n-1:
            if len(l_parcial) < 0.75*length and n!=1: # mantém o final se tiver mais de 75% do tamanho de um pedaço(length)
                continue
        def check_habilitacao(l):
            chunks = []
            sorted_habilitacao = sorted(lista_habilitacao.items(), key=lambda x: x[0])
            for key, value in sorted_habilitacao:
                if re.search(value, l):
                    chunks.append(1)
                else:
                    chunks.append(0)
            return chunks
        l_total.append(l)
        segment_label.append(check_habilitacao(l))
        id.append(id_document)
    return l_total, id, segment_label

In [ ]:
import tqdm as tqdm
def chunking(novodf):
    tqdm.tqdm.pandas()
    iter = iterDocuments(novodf.shape[0])
    result = novodf['text'].progress_apply(lambda text: get_text_split(text,iter))
    ids_documents = [_ for i in result for _ in i[1]]
    chunks = [_ for i in result for _ in i[0]]
    segment_label = [_ for i in result for _ in i[2]]
    bar = tqdm.tqdm(total=len(ids_documents))
    labels = novodf.loc[:, novodf.columns[1:]].values.tolist()
    labels_map = {i:_ for _,i in enumerate(novodf.columns[1:])}
    # Mapear para ordem alfabética
    
    multilabel = []
    for chunk,ids,seg_label in zip(chunks,ids_documents,segment_label):
        data = dict()
        data['text'] = chunk
        data['id_document'] = ids
        data['document_label'] = labels[ids]
        data['segment_label'] = seg_label
        data['labels_map'] = labels_map
        multilabel.append(data)
        bar.update(1)
    return multilabel

In [ ]:
import pandas as pd
#treino=pd.read_csv('dataset_bid_notices_weak_hab_clean_treino.csv')
#teste=pd.read_csv('dataset_bid_notices_weak_hab_clean_teste.csv')
#validacao=pd.read_csv('dataset_bid_notices_weak_hab_clean_validacao.csv')
sintetico = pd.read_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERT/Datasets/Indicios/dataset_syntetic.csv')
gold = pd.read_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERT/Datasets/Indicios/dataset_bid_notices_gold.csv')
#print(treino.shape)
#train=chunking(treino)
#print(teste.shape)
#test=chunking(teste)
#print(validacao.shape)
#validation=chunking(validacao)
print(sintetico.shape)
sintetic=chunking(sintetico)
print(gold.shape)
gold=chunking(gold)

In [ ]:
#pd.DataFrame(train).to_pickle('dataset_bid_notices_weak_hab_clean_treino_chunk_more.pkl')
#pd.DataFrame(test).to_pickle('dataset_bid_notices_weak_hab_clean_teste_chunk_more.pkl')
#pd.DataFrame(validation).to_pickle('dataset_bid_notices_weak_hab_clean_validacao_chunk_more.csv')
pd.DataFrame(sintetic).to_pickle('/var/projetos/Jupyterhubstorage/victor.silva/HelBERT/Datasets/Indicios/dataset_syntetic_chunk.pkl')
pd.DataFrame(gold).to_pickle('/var/projetos/Jupyterhubstorage/victor.silva/HelBERT/Datasets/Indicios/dataset_bid_notices_gold_chunk.pkl')

In [ ]:
validation

In [ ]:
train.to_csv('./train-BERT-indicios-weak-2/dataset_bid_notices_weak_hab_clean_treino_chunk.csv',index=False)
test.to_csv('./train-BERT-indicios-weak-2/dataset_bid_notices_weak_hab_clean_teste_chunk.csv',index=False)
validation.to_csv('./train-BERT-indicios-weak-2/dataset_bid_notices_weak_hab_clean_validacao_chunk.csv',index=False)